In [1]:
import cv2
import numpy as np
import os
import shutil
import sys

In [2]:
#matches is of (3|4 X 2 X 2) size. Each row is a match - pair of (kp1,kp2) where kpi = (x,y)
def get_transform(matches, is_affine):
    src_points, dst_points = matches[:,0], matches[:,1]

    if is_affine:
        T, _ = cv2.estimateAffinePartial2D(src_points, dst_points)
    else:
        T, _ = cv2.findHomography(src_points, dst_points, method=cv2.RANSAC)
    return T
	

def stitch(img1, img2):
	# Create a binary mask where img2 has non-zero pixels
	mask = (img2 > 0).any(axis=-1)  # Shape: (H, W)

	# Pad the mask to handle boundary conditions
	padded_mask = np.pad(mask, ((1, 1), (1, 1)), mode='constant', constant_values=0)

	# Check the neighbors in the 3x3 region
	neighbor_mask = (
			padded_mask[:-2, :-2] & padded_mask[:-2, 1:-1] & padded_mask[:-2, 2:] &  # Top row
			padded_mask[1:-1, :-2] & padded_mask[1:-1, 1:-1] & padded_mask[1:-1, 2:] &  # Middle row
			padded_mask[2:, :-2] & padded_mask[2:, 1:-1] & padded_mask[2:, 2:]  # Bottom row
	)

	# Combine the neighbor mask with the original mask
	valid_mask = neighbor_mask & mask

	# Use the valid mask to prioritize img2
	stitched_image = np.where(valid_mask[..., None], img2, img1)

	return stitched_image

# Output size is (w,h)
def inverse_transform_target_image(target_img, original_transform, output_size):
    if original_transform.shape == (2, 3):      # Affine transformation
        affine_transform = np.vstack([original_transform, [0,0,1]])
        inverse_transform = np.linalg.inv(affine_transform)[:2, :]

        warped_img = cv2.warpAffine(target_img, inverse_transform, output_size)
    elif original_transform.shape == (3, 3):    # Homography
        inverse_transform = np.linalg.inv(original_transform)
        warped_img = cv2.warpPerspective(target_img, inverse_transform, output_size)
    return warped_img

# returns list of pieces file names
def prepare_puzzle(puzzle_dir):
	edited = os.path.join(puzzle_dir, 'abs_pieces')
	if os.path.exists(edited):
		shutil.rmtree(edited)
	os.mkdir(edited)
	
	affine = 4 - int("affine" in puzzle_dir)
	
	matches_data = os.path.join(puzzle_dir, 'matches.txt')
	n_images = len(os.listdir(os.path.join(puzzle_dir, 'pieces')))

	matches = np.loadtxt(matches_data, dtype=np.int64).reshape(n_images-1,affine,2,2)
	
	return matches, affine == 3, n_images

In [3]:
def visualize_transform(original_img, warped_img):
    cv2.imshow("Original Image", original_img)
    cv2.imshow("Warped Image", warped_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [8]:
puzzle_dir = 'puzzles/puzzle_homography_1'
matches, is_affine, n = prepare_puzzle(puzzle_dir)
img1 = cv2.imread(os.path.join(puzzle_dir, 'pieces/piece_1.jpg'))

stitched_img = img1

height, width = img1.shape[:2]
output_size = (width, height)

for i in range(max(1, n - 2)):
    curr_matches = matches[i]
    target_img = cv2.imread(os.path.join(puzzle_dir, 'pieces', f'piece_{i+2}.jpg'))
    
    transform = get_transform(curr_matches, is_affine)
    warped_target = inverse_transform_target_image(target_img, transform, output_size)
    
    #DEBUG
    #visualize_transform(target_img, warped_target)

    stitched_img = stitch(stitched_img, warped_target)

cv2.imwrite('stitched_image.jpg', stitched_img)
cv2.imshow('Stitched Image', stitched_img)
cv2.waitKey(0)
cv2.destroyAllWindows()